In [1]:
import python_utils.python_utils.caching as caching
import ml_stuff_run.ml_stuff_run.constants as constants
caching.init(constants.cache_folder, constants.which_hash)
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mpl_colors
import matplotlib.cm as cm
import pdb
import numpy as np
import scipy
import ml_stuff.ml_stuff.hier_variational_regr as hvr
import causal_frl.causal_frl.hier_treatment_variational_regr as hvr

/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: mvtnorm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: Matrix

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: stats4

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: gmm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: Use

In [3]:
# random seed
np.random.seed(41)

In [4]:
# define params for simulating (x_ns, T_ns)
N = 5000
T_prob = 0.5
#K = 25
K=4
x_prob = 0.5

In [5]:
# simulate (x_ns, T_ns)
#x_ns = np.random.multivariate_normal(np.zeros(K), np.eye(K) * x_var, N)
x_ns = (np.random.uniform(size=(N,K)) < x_prob).astype(int)
T_ns = np.random.randint(0, 2, N)

In [6]:
# define rule_fs
print hvr
all_rule_fs = [hvr.binary_rule(k,1) for k in xrange(K)]
#rule_fs = [hvr.binary_rule(k,1) for k in xrange(K)] + [hvr.binary_rule(k,0) for k in xrange(K)]
print rule_fs
#rule_fs = [hvr.mem_address_rule(k,K) for k in xrange(K)]

<module 'causal_frl.causal_frl.hier_treatment_variational_regr' from '/Users/fultonw/Documents/projects/code/causal_frl/causal_frl/hier_treatment_variational_regr.py'>
[bin0_1, bin1_1, bin2_1, bin3_1]


In [7]:
# define p_y_ns_given_theta, theta
true_rule_f_ls = all_rule_fs[0:3] + [hvr.default_rule()]
#true_rule_f_ls = rule_fs[0:10:2] + [hvr.default_rule()]
#true_rule_f_ls = rule_fs
true_L = len(true_rule_f_ls)
true_p_y_ns_given_theta = hvr.p_y_ns_given_theta(true_rule_f_ls)
true_mu = np.ones(K)*5.0
true_prec = np.eye(K)/1.0
true_lambda_ls = np.ones(true_L) / 1.0
true_delta = np.cumsum(np.arange(1,true_L+1,1))
true_theta = hvr.theta(true_mu, true_prec, np.random.multivariate_normal(true_mu, true_prec, true_L), true_lambda_ls, true_delta)
print 'true delta:', true_delta
print 'true lambdas', true_lambda_ls
print 'true rule_f_ls', true_rule_f_ls
print 'supports', pd.Series(true_p_y_ns_given_theta.get_z_ns_num(x_ns)).value_counts()

true delta: [ 1  3  6 10]
true lambdas [ 1.  1.  1.  1.]
true rule_f_ls [bin0_1, bin1_1, bin2_1, default]
supports 0    2574
1    1215
3     638
2     573
dtype: int64


In [8]:
# simulate y_ns
y_ns = true_p_y_ns_given_theta.sample(true_theta, (x_ns,T_ns))

In [9]:
# define p_theta
v_0 = 66.0001
assert v_0 > K-1
T_0 = 1.0 * np.eye(K) 
c_0 = .0001 
m_0 = np.zeros(K)
alpha_0 = 1.
beta_0 = 1.
r_0 = -1000.
s_0 = 1000.
p_theta = hvr.p_theta(v_0, T_0, c_0, m_0, alpha_0, beta_0, r_0, s_0)

In [10]:
# define get_posterior_f
recovery_num_iters = 15
recovery_get_posterior_f = hvr.get_posterior_f(recovery_num_iters)

In [11]:
#for true model, try to recover true theta
true_model = hvr.model(p_theta, true_p_y_ns_given_theta)
v, T, c, m, mu_B_ls, prec_B_ls, alpha_lambda_ls, beta_lambda_ls, mu_delta, prec_delta = recovery_get_posterior_f(true_model, (x_ns, T_ns), y_ns)
print 'v',v
print 'T',T 
print 'c',c
print 'm',m
print 'mu_B_ls', mu_B_ls
print 'prec_B_ls', prec_B_ls
print 'alpha_lambda_ls', alpha_lambda_ls
print 'beta_lambda_ls', beta_lambda_ls
print 'mu_delta', mu_delta
print 'prec_delta',prec_delta
print 'mu_lambda_ls', alpha_lambda_ls / beta_lambda_ls

v 70.0001
T [[ 0.9305402   0.01180504  0.08144274 -0.03060717]
 [ 0.01180504  0.95836017 -0.05244916  0.04186341]
 [ 0.08144274 -0.05244916  0.64547799  0.22040556]
 [-0.03060717  0.04186341  0.22040556  0.57597806]]
c 4.0001
m [ 2.96460495  3.72926249  5.7555861   4.22150463]
mu_B_ls [[ 3.13007924  3.65170523  5.22703972  4.3343088 ]
 [ 2.87386118  3.85183523  6.46293995  3.39212671]
 [ 2.99197193  3.69987917  5.64496805  4.74785725]
 [ 2.97687061  3.71715944  5.68837907  4.41091417]]
prec_B_ls [array([[ 2642.66846778,  1303.61083679,  1294.46628025,  1274.60631575],
       [ 1303.61083679,  1369.86979061,   657.23382545,   660.83159977],
       [ 1294.46628025,   657.23382545,  1333.94880404,   663.31585186],
       [ 1274.60631575,   660.83159977,   663.31585186,  1317.06734247]]), array([[  6.51379073e+01,   8.26353794e-01,   5.70099999e+00,
         -2.14250502e+00],
       [  8.26353794e-01,   1.28932536e+03,   6.21029024e+02,
          6.79940775e+02],
       [  5.70099999e+00, 

In [12]:
# define list of models to calculate evidence for
candidate_models = [hvr.model. model_from_p_theta_and_rule_fs(p_theta, list(rule_fs)) for rule_fs in hvr.subsequence_iterator(all_rule_fs)]
print len(candidate_models)

64


In [13]:
# define function to calculate evidence lower bound
num_iters = 5
get_posterior_f = hvr.get_posterior_f(num_iters)
get_evidence_f = hvr.get_evidence_f(get_posterior_f, hvr.log_evidence_lower_bound)

In [18]:
# calculate evidence for each model
results = []

@basic.do_cprofile
def run():
    for candidate_model in candidate_models:
        print candidate_model
        results.append([repr(candidate_model),get_evidence_f(candidate_model, (x_ns, T_ns), y_ns)])
run()
print results

[bin0_1, default]
[bin1_1, default]
[bin2_1, default]
[bin3_1, default]
[bin0_1, bin1_1, default]
[bin0_1, bin2_1, default]
[bin0_1, bin3_1, default]
[bin1_1, bin0_1, default]
[bin1_1, bin2_1, default]
[bin1_1, bin3_1, default]
[bin2_1, bin0_1, default]
[bin2_1, bin1_1, default]
[bin2_1, bin3_1, default]
[bin3_1, bin0_1, default]
[bin3_1, bin1_1, default]
[bin3_1, bin2_1, default]
[bin0_1, bin1_1, bin2_1, default]
[bin0_1, bin1_1, bin3_1, default]
[bin0_1, bin2_1, bin1_1, default]
[bin0_1, bin2_1, bin3_1, default]
[bin0_1, bin3_1, bin1_1, default]
[bin0_1, bin3_1, bin2_1, default]
[bin1_1, bin0_1, bin2_1, default]
[bin1_1, bin0_1, bin3_1, default]
[bin1_1, bin2_1, bin0_1, default]
[bin1_1, bin2_1, bin3_1, default]
[bin1_1, bin3_1, bin0_1, default]
[bin1_1, bin3_1, bin2_1, default]
[bin2_1, bin0_1, bin1_1, default]
[bin2_1, bin0_1, bin3_1, default]
[bin2_1, bin1_1, bin0_1, default]
[bin2_1, bin1_1, bin3_1, default]
[bin2_1, bin3_1, bin0_1, default]
[bin2_1, bin3_1, bin1_1, default]
[bin

In [15]:
# print evidence for each model
print len(results)
display_html(pd.DataFrame.from_records(results, columns=['rules','log evidence']).sort(columns=['log evidence']).to_html(), raw=True)

64


,rules,log evidence
3,"[bin3_1, default]",-27967.736812
15,"[bin3_1, bin2_1, default]",-27635.170397
13,"[bin3_1, bin0_1, default]",-27630.297342
14,"[bin3_1, bin1_1, default]",-27610.046463
35,"[bin3_1, bin0_1, bin2_1, default]",-27520.648181
37,"[bin3_1, bin1_1, bin2_1, default]",-27483.550296
59,"[bin3_1, bin0_1, bin2_1, bin1_1, default]",-27408.972169
39,"[bin3_1, bin2_1, bin1_1, default]",-27364.634732
38,"[bin3_1, bin2_1, bin0_1, default]",-27338.726185
61,"[bin3_1, bin1_1, bin2_1, bin0_1, default]",-27249.876724


In [16]:
# define simulated annealing model fitter
SA_num_iters = 1500
temperature = 1.
def reject_f(model,(x_ns,T_ns), y_ns): 
    return np.sum(model.p_y_ns_given_theta.get_z_ns(x_ns), axis=0).min() < 10
model_fitter = hvr.simulated_annealing_model_fitter(all_rule_fs, p_theta, SA_num_iters, temperature, get_evidence_f, reject_f)

In [17]:
# run simulated annealing
#best_model, best_evidence = model_fitter((x_ns, T_ns), y_ns)
#print best_model, best_evidence